## \#0 Imports

In [9]:
import sqlite3
import pandas as pd

In [13]:
conn = sqlite3.connect('melee_player_database\melee_player_database.db')

cursor = cnx.cursor()

# cursor.execute('.tables')
cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")


# Fetch all the table names
tables = cursor.fetchall()

# Print the table names
print("Tables in the database:")
for table in tables:
    print(table[0])

# Close the cursor and connection
cursor.close()
conn.close()

Tables in the database:
tournament_info
sets
players
ranking
ranking_seasons
